# <span style="color:blue"> Premier League Bar Chart Vizualization </span>

Goal: create a bar chart race of the premier leauge temas throughout the season

Obejectives:
1. Loop through the matchweeks and gather the data
2. Format the data in excel
3. Code the bar chart

Strategy:
1. Loop through the matchweeks and collect team name aswell as points in the leauge


In [1]:
#Import the libraries
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By


from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument("--headless")

import pandas as pd
import time

from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup

### 1. Loop through the Matchweeks

In [2]:
#Open the page

#Open chrome tab
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

#Open prem league 
url = 'https://www.premierleague.com/tables?co=1&se=489&ha=-1'
driver.get(url)



#wait for the page to load
wait = WebDriverWait(driver, 10) 
element =wait.until(EC.visibility_of_element_located((By.XPATH ,'//*[@id="onetrust-accept-btn-handler"]' )))
#click Allow cookies
element.send_keys(Keys.ENTER)

#Close a popup
try:
    driver.find_element(By.XPATH,'//*[@id="advertClose"]').click()

except:
    pass

In [3]:
# Click and open matchweek dropdown

matchweek_btn = driver.find_element(By.XPATH,'//*[@id="mainContent"]/div[2]/div[1]/div[1]/section/div[3]/div[2]')
#matchweek_btn.click()
# matweek dropdowns between 154 and 154+28
match_weeks = driver.find_elements(By.TAG_NAME,'li')[154:182]



In [4]:
#Create lists to contain all the data
points = []
teams = []


#Loop through all the matchweeks

for i in match_weeks:
    time.sleep(2)
    matchweek_btn.click()
    time.sleep(1)
    i.click()
    
    time.sleep(2)
    #Get the table data
    table = [ i.text for i in driver.find_elements(By.TAG_NAME,'td')]

    #Get rid of the none values
    new_list = list(filter(lambda x: x != '', table))
    points_weekx = new_list[9::10]
    teams_weekx = new_list[1::10]

    #Append data to lists
    points.append(points_weekx)
    teams.append(teams_weekx)
    
    
    

### 2. Data Manipulation

In [5]:
import numpy as np

In [6]:
#Need to delete the first elements because its included at the end
del points[0]
del teams[0]

In [41]:
df = pd.DataFrame(columns = ['Date','Teams','Points'])

p = [i for row in points for i in row] 
t = [i for row in teams for i in row] 

df['Points'] = p
df['Teams'] = t
df

,Date,Teams,Points
0,NaN,Tottenham Hotspur,3
1,NaN,Bournemouth,3
2,NaN,Arsenal,3
3,NaN,Manchester City,3
4,NaN,Newcastle United,3
...,...,...,...
435,NaN,Nottingham Forest,26
436,NaN,Leicester City,25
437,NaN,West Ham United,24
438,NaN,Bournemouth,24


In [54]:
from datetime import datetime

start_date = datetime.strptime("2022-08-06", "%Y-%m-%d")
date_list = pd.date_range(start_date, periods=26, freq='W')

In [56]:
date_list

DatetimeIndex(['2022-08-07', '2022-08-14', '2022-08-21', '2022-08-28',
               '2022-09-04', '2022-09-11', '2022-09-18', '2022-09-25',
               '2022-10-02', '2022-10-09', '2022-10-16', '2022-10-23',
               '2022-10-30', '2022-11-06', '2022-11-13', '2022-11-20',
               '2022-11-27', '2022-12-04', '2022-12-11', '2022-12-18',
               '2022-12-25', '2023-01-01', '2023-01-08', '2023-01-15',
               '2023-01-22', '2023-01-29'],
              dtype='datetime64[ns]', freq='W-SUN')

In [71]:
i =1
z = 0
for x in range(0,22):
    
    
    i +=20
    df.iloc[z:i,0] = date_list[x]
    z+=20
    

In [72]:
df

,Date,Teams,Points
0,2022-08-07 00:00:00,Tottenham Hotspur,3
1,2022-08-07 00:00:00,Bournemouth,3
2,2022-08-07 00:00:00,Arsenal,3
3,2022-08-07 00:00:00,Manchester City,3
4,2022-08-07 00:00:00,Newcastle United,3
...,...,...,...
435,2023-01-01 00:00:00,Nottingham Forest,26
436,2023-01-01 00:00:00,Leicester City,25
437,2023-01-01 00:00:00,West Ham United,24
438,2023-01-01 00:00:00,Bournemouth,24


### 3. Extract the data
- Save the df to a csv
- Manipulate in excel

In [74]:
df.to_excel('PL-accumulative points.xlsx')

### 4. Vizualize the data
- run the below code to run the viz

In [1]:
from sjvisualizer import DataHandler, Canvas, BarRace, StaticImage
import json
import time

In [2]:
FPS = 60
DURATION = 0.3
EXCEL_FILE = 'PL-accumulative points.xlsx'

In [3]:
# load the data into a data frame
df = DataHandler.DataHandler(excel_file=EXCEL_FILE, number_of_frames=FPS*DURATION*60).df

# creating the canvas
canvas = Canvas.canvas()

# adding the racing bar chart
bar_chart = BarRace.bar_race(df=df, canvas=canvas.canvas)
canvas.add_sub_plot(bar_chart)

# adding a title
canvas.add_title("Premier Leugue Vizualisation", color=(0,0,0))
canvas.add_sub_title("From Week 1 - Week 28", color=(150,150,150))

# adding a time
canvas.add_time(df=df, time_indicator="year")

#Give time to reecord
#time.sleep(5)

# play the animation
canvas.play(fps=FPS)

loading new data frame
Perparing data
Working on 0/22
Working on 1/22
2022-08-07 03:06:40
2022-08-14 00:00:00
Working on 2/22
2022-08-14 03:06:40
2022-08-21 00:00:00
Working on 3/22
2022-08-21 03:06:40
2022-08-28 00:00:00
Working on 4/22
2022-08-28 03:06:40
2022-09-04 00:00:00
Working on 5/22
2022-09-04 03:06:40
2022-09-11 00:00:00
Working on 6/22
2022-09-11 03:06:40
2022-09-18 00:00:00
Working on 7/22
2022-09-18 03:06:40
2022-09-25 00:00:00
Working on 8/22
2022-09-25 03:06:40
2022-10-02 00:00:00
Working on 9/22
2022-10-02 03:06:40
2022-10-09 00:00:00
Working on 10/22
2022-10-09 03:06:40
2022-10-16 00:00:00
Working on 11/22
2022-10-16 03:06:40
2022-10-23 00:00:00
Working on 12/22
2022-10-23 03:06:40
2022-10-30 00:00:00
Working on 13/22
2022-10-30 03:06:40
2022-11-06 00:00:00
Working on 14/22
2022-11-06 03:06:40
2022-11-13 00:00:00
Working on 15/22
2022-11-13 03:06:40
2022-11-20 00:00:00
Working on 16/22
2022-11-20 03:06:40
2022-11-27 00:00:00
Working on 17/22
2022-11-27 03:06:40
2022-1

FPS: 41
FPS: 32
FPS: 41
FPS: 43
FPS: 50
FPS: 41
FPS: 36
FPS: 49
FPS: 49
FPS: 44
FPS: 39
FPS: 41
FPS: 49
FPS: 47
FPS: 27
FPS: 42
FPS: 49
FPS: 56
FPS: 23
FPS: 59
FPS: 46
FPS: 41
FPS: 33
FPS: 56
FPS: 48
FPS: 49
FPS: 45
FPS: 49
FPS: 44
FPS: 46
FPS: 38
FPS: 41
FPS: 49
FPS: 48
FPS: 43
FPS: 49
FPS: 49
FPS: 54
FPS: 52
FPS: 37
FPS: 32
FPS: 41
FPS: 49
FPS: 32
FPS: 50
FPS: 56
FPS: 46
FPS: 47
FPS: 33
FPS: 55
FPS: 50
FPS: 50
FPS: 44
FPS: 35
FPS: 49
FPS: 50
FPS: 32
FPS: 44
FPS: 54
FPS: 49
FPS: 30
FPS: 54
FPS: 49
FPS: 48
FPS: 49
FPS: 42
FPS: 48
FPS: 60
FPS: 54
FPS: 50
FPS: 44
FPS: 45
FPS: 59
FPS: 55
FPS: 49
FPS: 45
FPS: 55
FPS: 49
FPS: 50
FPS: 50
FPS: 32
FPS: 54
FPS: 50
FPS: 49
FPS: 32
FPS: 51
FPS: 49
FPS: 32
FPS: 31
FPS: 55
FPS: 48
FPS: 49
FPS: 30
FPS: 49
FPS: 55
FPS: 59
FPS: 40
FPS: 32
FPS: 30
FPS: 55
FPS: 30
FPS: 38
FPS: 46
FPS: 50
FPS: 33
FPS: 43
FPS: 55
FPS: 49
FPS: 48
FPS: 30
FPS: 55
FPS: 49
FPS: 50
FPS: 43
FPS: 56
FPS: 49
FPS: 47
FPS: 50
FPS: 25
FPS: 53
FPS: 54
FPS: 50
FPS: 47
FPS: 32
FPS: 33
